# GNSS Timeseries Project

## Internal Code

### Importing Packages

In [ ]:
import folium as fol
import pandas as pd
import numpy as np
import ipywidgets as wg
from ipywidgets import HBox, VBox
import matplotlib.pyplot as plt
from matplotlib import gridspec
import math
import geopy.distance
import json
import os
from os.path import exists
from os import makedirs
from datetime import datetime
import random
import urllib.request
import requests
from pathlib import Path
from earthscope_sdk.auth.device_code_flow import DeviceCodeFlowSimple
from earthscope_sdk.auth.auth_flow import NoTokensError

### Setting up Directories

In [ ]:
orglist = ["UNR", "JPL", "UNAV"]

def make_if_absent(folderpath):
    if not exists(folderpath):
        makedirs(folderpath)

make_if_absent("data")
for orgname in orglist:
    make_if_absent("data/"+orgname+"/")

### Reading from JSON file

In [ ]:
data_of = {}

for org in orglist:
    filepath = "data/" + org + "_data.json"
    if exists(filepath):
        with open(filepath, "r") as data_file:
            data_of[org] = json.load(data_file)

### Linear Detrend Function

In [ ]:
def FitTS(xdata,ydata,sig):
    '''Fit times series.  Linear trend currently
    data is xdata time in days, ydata (NEU), sig 
    https://en.wikipedia.org/wiki/Generalized_least_squares
    https://en.wikipedia.org/wiki/Reduced_chi-squared_statistic
    Outputs:
    array([grad, y-int]), [grad, grad_sd], [Wrms, chi_sq, ndata]
    '''
        
    yr = xdata/365.25; # Years Since 2000/1/1
    OmInv = 1./sig**2
    ndata = int(xdata.size)  # number of data points
    X = np.vstack((np.ones(ndata),yr)).T
    Bvec = (X.T * OmInv) @ ydata.T
    MCov = np.linalg.inv((X.T * OmInv) @ X)
    MEst = MCov @ Bvec
    Res = ydata - X @ MEst

    chi = np.sqrt((Res.T @ (Res*OmInv))/(ndata-2))
    wrms = np.sqrt(ndata/np.sum(OmInv))*chi
    
    return np.flip(MEst), [MEst[1],np.sqrt(MCov[1,1])], [wrms,chi,ndata]

def detrended(xdata, ydata, sig):
    m, c = FitTS(xdata, ydata, sig)[0]
    return ydata - (m * xdata / 365.25 + c)

### Timeseries Readers

In [ ]:
## FETCH TIMESERIES DATA - UNR 

SigLim = (10,10,30) # standard deviation

# UNR Data Fetcher
def Read_UNR(site, Update):
    # !! check for if site has no network
    site_info = data_of["UNR"][site]

    file_directory = "./data/UNR/"+site.upper()+"."+site_info["networks"][0]+".csv"
    file_name = site.upper()+"."+site_info["networks"][0]+".tenv3"
    fetch_url="http://geodesy.unr.edu/gps_timeseries/tenv3/plates/"+site_info["networks"][0]+"/"+file_name

    if exists(file_directory) and not bool(Update) :
        print('Reading from file ',file_directory)
        df = pd.read_csv(file_directory,delimiter=',')
    else:
        print('Downloading from ',fetch_url)
        df = pd.read_csv(fetch_url,delimiter=r"\s+",header=1)
        df.to_csv(file_directory,index=False)

    ninit = len(df)
    # Remove data with NEU sigmas > 10, 10 and 30 mm.
    # Apply sequencially because 'and' does not seem to work
    if SigLim[0] > 0:
        df=df[df.iloc[:,14]<SigLim[0]/1000]
        df=df[df.iloc[:,15]<SigLim[1]/1000]
        df=df[df.iloc[:,16]<SigLim[2]/1000]
        print("Number after >{} {} {} mm NEU sigma removal".format(*SigLim),len(df),"Read",ninit)
    npdat = df.to_numpy()

    # Extract time 
    t=list(npdat[:,1])
    nd=list((npdat[:,10]-npdat[0,10])*1000) # remove first value so starts at zero
    # the same a UNAV. (Problem if times of first data point are different)
    ed=list((npdat[:,8]-npdat[0,8])*1000)
    ud=list((npdat[:,12]-npdat[0,12])*1000) 
    ns=list(npdat[:,15]*1000) ; es=list(npdat[:,14]*1000) ; us=list(npdat[:,16]*1000)

    n = 0
    to = []
    td = np.zeros(len(t))
    for v in t:
        to = np.append(to, datetime.strptime(v, '%y%b%d'))
        dt = to[n] - datetime(2000, 1, 1,0,0)  # Time difference from 2000/1/1
        td[n] = dt.total_seconds()/86400.  # Days from 2000/1/1
        n += 1

    tseries = [td, nd, ns, ed, es, ud, us]
    times = to
    return times, tseries

def Read_JPL(site, Update):
    # !! check for if site has no network
    csv_file_directory = "./data/JPL/"+site+".csv"
    
    fetch_url="https://sideshow.jpl.nasa.gov/pub/JPL_GPS_Timeseries/repro2018a/post/point/"+site+".series"

    if exists(csv_file_directory) and not bool(Update):
        print('Reading from file',csv_file_directory)
        df = pd.read_csv(csv_file_directory,delimiter=',')
    else:
        print('Downloading from ',fetch_url)
        df = pd.read_csv(fetch_url,delimiter=r"\s+")
        df.to_csv(csv_file_directory,index=False)

    ninit = len(df)
    # Remove data with NEU sigmas > 10, 10 and 30 mm.
    # Apply sequencially because 'and' does not seem to work
    if SigLim[0] > 0:
        df=df[df.iloc[:,5]<SigLim[0]/1000]
        df=df[df.iloc[:,4]<SigLim[1]/1000]
        df=df[df.iloc[:,6]<SigLim[2]/1000]
        print("Number after >{} {} {} mm NEU sigma removal".format(*SigLim),len(df),"Read",ninit)

    npdat = df.to_numpy()

    nd=list((npdat[:,2]-npdat[0,2])*1000) # 3rd column, remove first value so starts at zero
    # the same a UNAV. (Problem if times of first data point are different)
    ed=list((npdat[:,1]-npdat[0,1])*1000) # 2nd column
    ud=list((npdat[:,3]-npdat[0,3])*1000) # 4th column
    ns=list(npdat[:,5]*1000) ; es=list(npdat[:,4]*1000) ; us=list(npdat[:,6]*1000) # what do any of these meannnn
    to = np.empty(len(npdat[:,10]), dtype = object)

    for i in range(len(to)):
        each_date = datetime((int(npdat[:,11][i])), int((npdat[:,12][i])), int((npdat[:,13][i])))
        to[i] = each_date

    td_counter = 0
    td = np.zeros(len(npdat[:,10]))
    for time_value in npdat[:,10]:
        td[td_counter] = time_value/31557600 # days from 2000/1/1
        td_counter += 1

    tseries = [td, nd, ns, ed, es, ud, us]
    times = to
    # print(to)
    return times, tseries


def get_es_file(url, directory_to_save_file='./', token_path='./'):
    """function to get earthscope data using es-sdk

    Parameters
    ----------
    url : string
        url of desired file at gage-data.earthscope.org
    directory_to_save_file : str, optional
        path of directory in which to save the file, by default cwd
    token_path : str, optional
        path of directory in which to save the token, by default './'
    """
    # instantiate the device code flow subclass
    device_flow = DeviceCodeFlowSimple(Path(token_path))
    try:
      # get access token from local path
      device_flow.get_access_token_refresh_if_necessary()
    except NoTokensError:
      # if no token was found locally, do the device code flow
      device_flow.do_flow()
    token = device_flow.access_token

    # request a file and provide the token in the Authorization header
    file_name = Path(url).name

    r = requests.get(url, headers={"authorization": f"Bearer {token}"})
    if r.status_code == requests.codes.ok:
      # save the file
      with open(Path(Path(directory_to_save_file) / file_name), 'wb') as f:
        for data in r:
            f.write(data)
    else:
      #problem occured
      print(f"failure: {r.status_code}, {r.reason}")


def Read_UNAV(site, Update):
    '''Function to read data from UNAVCO and return numpy array and datetime arrays.
    Input:
       sites - 4-character site name
       Update - 1 Re-read data from server; 0 Use existing copy
    Output
       times   - date-times dates thar can be used in plotting
       tseries - time series with days for 2000/1/1, dn,sn, de,se, du, su'''
    global SigLim
    # Incase of token refresh problems may need
    #!es sso refresh

    # Create the file name and URL
    filen = site.upper()+".cwu.nam14.csv"
    whurl="https://data.unavco.org/archive/gnss/products/position/"+site.upper()+"/"+filen

    #PBO Station Position Time Series.
    #Format Version, 1.2.0
    #Reference Frame, NAM14
    #4-character ID, P177
    #Station name, CoDeTierraCN2008
    #Begin Date, 2008-05-09
    #End Date, 2021-10-06
    #Release Date, 2021-10-07
    #Source file, P177.cwu.nam14.pos
    #Offset from source file, 166.01 mm North, -139.45 mm East, -3.62 mm Vertical
    #Reference position, 37.5281683684 North Latitude, -122.4950535711 East Longitude, 71.79172 meters elevation
    #Date, North (mm), East (mm), Vertical (mm), North Std. Deviation (mm), East Std. Deviation (mm), Vertical Std. Deviation (mm), Quality,
    #2008-05-09,0.00, 0.00, 0.00, 2.08, 1.66, 7.84, repro,
    #2008-05-10,0.38, 0.83, 2.66, 2.13, 1.7, 7.99, repro,
    #...
    # print(filen, whurl)
    if exists(filen) and not bool(Update):
        print('Reading from file',filen)
        df = pd.read_csv(filen,delimiter=',',header=11)
    else:
        print('Downloading from ',whurl)
        get_es_file(whurl)
        df = pd.read_csv(filen,delimiter=',',header=11)
        # Write csv local copy for later use
        #print('Creating',filen)
        #df.to_cvs(filen)
        # df.to_csv(filen,index=False)

    ninit = len(df)
    # Remove data with NEU sigmas > 10, 10 and 30 mm.
    # Apply sequencially because 'and' does not seem to work (latter use values
    # extracted from GUI box)
    if SigLim[0] > 0 :
        df=df[df.iloc[:,4]<SigLim[0]];
        df=df[df.iloc[:,5]<SigLim[1]];
        df=df[df.iloc[:,6]<SigLim[2]];
        print("Number after >{} {} {} mm NEU sigma removal".format(*SigLim),len(df),"Read",ninit)

    npdat =df.to_numpy()
    # Extract time,and data and sigma
    t=list(npdat[:,0]);nd=list(npdat[:,1]); ed=list(npdat[:,2]); ud=list(npdat[:,3]);
    ns=list(npdat[:,4]) ; es=list(npdat[:,5]) ; us=list(npdat[:,6])
    # Now convert the time into a datetime type (to) and days from 2000/01/01
    # as float array (td).
    # We can use 'to' in plotting and 'td' for fitting to data
    n = 0
    to = []
    td = np.zeros(len(t))
    for v in t:
        to = np.append(to,datetime.strptime(v, '%Y-%m-%d'))
        dt = to[n] - datetime(2000, 1, 1)  # Time difference from 2000/1/1
        td[n] = dt.total_seconds()/86400.  # Days from 2000/1/1
        n += 1

    # Now construct data array
    tseries = np.array([td,nd,ns,ed,es,ud,us])
    times = to
    return times, tseries


Read = {
        "UNR": Read_UNR,
        "JPL": Read_JPL,
        "UNAV": Read_UNAV
    }

### Map Functions

In [ ]:
## GENERAL MAP OPERATIONS

d_km = lambda coords1, coords2: geopy.distance.geodesic(coords1,coords2).km
d_mi = lambda coords1, coords2: geopy.distance.geodesic(coords1,coords2).miles

def plot_locations(map, coordslist, tooltiplist, color="blue", dotrad = 2):
    """
    map: Folium map
    coordslist: [(lat1, lon1), (lat2,lon2), ...]
    tooltip: [tag1, tag2, ...]
    """
    for coords,tooltip in zip(coordslist,tooltiplist):
        fol.CircleMarker(
                location=coords,
                tooltip=tooltip,
                radius = dotrad,
                stroke = False,
                fill_color = color,
                fill_opacity = 1,
                opacity = 1,
                #icon=fol.Icon(icon="globe"),
            ).add_to(map)


def nearby_sites(site_data, site, radius_in_km):
    """
    site = ID of a site OR coordinates provided as a list
    radius_in_km = radius within which we want to plot sites
    Returns: [ (neighbor_property, lat, lon, distance from site), ... ]
    """
    if site[0] in "([": # if it's a coordinate pair instead of a site id
        site = list(eval(site))
        
    cur_coords = site

    if isinstance(site,str):
        cur_coords = site_data[site]["location"]

    assert not isinstance(cur_coords[0],str) # by now cur_coords is a coordinate pair regardless of input

    answer = []

    for siteid in site_data:
        try:
            site_coords = site_data[siteid]["location"]
            distance = d_km(site_coords, cur_coords)
            if distance < radius_in_km and site != siteid:
                answer.append((siteid,*site_coords,distance))
        except: pass

    return answer, cur_coords # offset specifies whether we include nearest site or not (depends on if we input a site or coordinate pair)


def basic_circle(map, center, radius_in_km):
    fol.Circle(
        location=center,
        radius=radius_in_km * 1000, # in metres
        color="black",
        weight=1,
        fill_opacity=0.0,
        opacity=1,
        fill_color="green",
        fill=True,  # gets overridden by fill_color
        #tooltip="I am in meters",
    ).add_to(map)

def vec_add(c1,c2):
    return [c1[0]+c2[0], c1[1]+c2[1]]

def draw_vector(map, center, direction, scale, color="black"):
    dvec = [direction[0] * scale * abs(math.cos(center[0] * math.pi/180 )), direction[1] * scale]
    endpoint = vec_add(center, dvec)
    fol.PolyLine(locations=[center, endpoint],weight=2,color = color).add_to(map)

### Timeseries Functions

In [ ]:
def remove_brac(string):
    '''
    "P123 (UNR)" --> ("P123", "UNR")
    '''
    i=-1
    while string[i] != '(': i-=1
    return string[:i-1], string[i+1:-1]


def plot_ts_graph(siteid, org, Update, detrend, ax0, ax1, ax2, yearrange, errorbars):
    '''
    Adding a single plot to the original plot defined in plot_ts_graph_list()
    '''
    
    times, tseries = Read[org](siteid, Update)
    td, nd, ns, ed, es, ud, us = tseries

    ## restricting timeseries to yearrange
    # find the start and end index
    start, end = 0, len(td)
    while times[start] < yearrange[0]: start+=1
    while times[end-1] > yearrange[1]: end-=1
    # restrict
    times, td = times[start:end], td[start:end]
    nd, ns = nd[start:end], ns[start:end]
    ed, es = ed[start:end], es[start:end]
    ud, us = ud[start:end], us[start:end]

    if detrend: 
        nd = detrended(np.array(td), np.array(nd), np.array(ns))
        ed = detrended(np.array(td), np.array(ed), np.array(es))
        ud = detrended(np.array(td), np.array(ud), np.array(us))
    
    if errorbars:
        # ax0.errorbar(times,nd,yerr=[ns,ns],errorevery=1, ecolor='black')
        # ax1.errorbar(times,ed,yerr=[es,es],errorevery=1, ecolor='black')
        # ax2.errorbar(times,ud,yerr=[us,us],errorevery=1, ecolor='black')
        ax0.fill(list(times) + list(reversed(times)), 
                 list(np.array(nd)+np.array(ns)) + list(np.flip(np.array(nd)-np.array(ns))), 
                 alpha=0.4, linewidth=3, label="_"+siteid)
        ax1.fill(list(times) + list(reversed(times)), 
                 list(np.array(ed)+np.array(es)) + list(np.flip(np.array(ed)-np.array(es))), 
                 alpha=0.4, linewidth=3, label="_"+siteid)
        ax2.fill(list(times) + list(reversed(times)), 
                 list(np.array(ud)+np.array(us)) + list(np.flip(np.array(ud)-np.array(us))), 
                 alpha=0.4, linewidth=3, label="_"+siteid)
    
    ax0.plot(times,nd, linewidth=0.5, label=siteid + " (" + org + ")")
    ax1.plot(times,ed, linewidth=0.5, label=siteid + " (" + org + ")")
    ax2.plot(times,ud, linewidth=0.5, label=siteid + " (" + org + ")")


def plot_ts_graph_list(idlist, Update, yearrange, detrend = False, errorbars = False, resolution = "Low Res"):
    '''
    Plots the timeseries of a list of sites.
    '''
    plt.figure(figsize=(15, 10), dpi = {"HD": 600, "Regular": 300, "Low Res": 100}[resolution])
    gs = gridspec.GridSpec(3, 1, height_ratios=[1, 1, 1]) 

    ax0 = plt.subplot(gs[0])
    ax0.set_ylabel("ΔNorth")
    ax0.yaxis.set_tick_params(labelrotation=90)
    plt.setp(ax0.get_xticklabels(), visible=False)
    
    ax1 = plt.subplot(gs[1], sharex = ax0)
    ax1.set_ylabel("ΔEast")
    ax1.yaxis.set_tick_params(labelrotation=90)
    plt.setp(ax1.get_xticklabels(), visible=False)

    ax2 = plt.subplot(gs[2], sharex = ax0)
    ax2.set_ylabel("ΔUp")
    ax2.yaxis.set_tick_params(labelrotation=90)
    ax2.set_xlabel("Time")

    for idorg in idlist:
        plot_ts_graph(*remove_brac(idorg), Update, detrend, ax0, ax1, ax2, yearrange, errorbars)

    ax0.legend(bbox_to_anchor=(1, 1))
    
    plt.subplots_adjust(hspace=.0)
    plt.show()

### Widgets

#### Fetcher Widgets

In [ ]:
# Buttons
update_UNR_butt     = wg.Button(description="Latest UNR", icon = "download")
update_JPL_butt     = wg.Button(description="Latest JPL", icon = "download")
update_UNAV_butt    = wg.Button(description="Latest UNAV", icon = "download")
# Outputs
fetcher_output      = wg.Output()

#### Availability Widgets

In [ ]:
# Inputs
site_searchbar      = wg.Text(value=None,placeholder='Site ID',description='Check Availability:',disabled=False, style= {'description_width': 'initial'})
org_avail_select    = wg.Dropdown(options=orglist+['other'], value='UNR', disabled=False, layout=wg.Layout(width='100px'))
site_search_submit  = wg.Button(description="Search!", icon = "search")
clear_log_butt      = wg.Button(description="Clear Log", icon = "times")
# Style
site_search_submit.style.button_color = 'rgb(196,253,196)'
clear_log_butt.style.button_color = 'mistyrose'
# Outputs
availability_output = wg.Output()

#### Map Widgets

In [ ]:
# Inputs
map = ["Not initialized yet!"] # allow direct editing of the map via entry mutation
layout              = lambda w: wg.Layout(width=w, height='40px')
new_map_butt        = wg.Button(description="Clear / New Map", icon = "map")
reload_map_butt     = wg.Button(description="Reload Map", icon = "refresh")
close_map_butt      = wg.Button(description="Close Map", icon = "times")
site_center         = wg.Text(value=None,placeholder='Site ID',description='Site:',disabled=False, style= {'description_width': 'initial'}, layout=layout("150px"))
org_map_select      = wg.Dropdown(options=orglist+['other'], value='UNR', disabled=False, layout=wg.Layout(width='100px'))
site_radius         = wg.Text(value=None,placeholder='in km',description='Radius:',disabled=False, style= {'description_width': 'initial'}, layout=layout("150px"))
site_circle_submit  = wg.Button(description="Add Plot!", icon = "map-marker")
plot_graph_butt     = wg.Button(description="Plot Distances", icon = "line-chart")
enter_code_form     = wg.Text(value=None, placeholder='Not Implemented Yet')
plot_vec_check      = wg.Checkbox(description="Plot Velocities", value=False)
# Style
new_map_butt.style.button_color = 'rgb(196,253,196)'
reload_map_butt.style.button_color = 'oldlace'
close_map_butt.style.button_color = 'mistyrose'
site_circle_submit.style.button_color = 'paleturquoise'
plot_graph_butt.style.button_color = 'paleturquoise'
# Outputs
map_output          = wg.Output()
nearest_site_output = wg.Output()
graph_output        = wg.Output()

#### Timeseries Widgets

In [ ]:
# Inputs
ts_site_form        = wg.Text(value=None,placeholder='Site ID',disabled=False, style= {'description_width': 'initial'}, layout=layout("150px"))
org_ts_select       = wg.Dropdown(options=orglist+['other'], value='UNR', disabled=False, layout=wg.Layout(width='100px'))
append_butt         = wg.Button(description="Add to List", icon = "plus-square")
plot_ts_butt        = wg.Button(description="Plot", icon = "line-chart")
plot_ts_res         = wg.Dropdown(options=['HD', 'Regular', 'Low Res'], value='Low Res', disabled=False, layout=wg.Layout(width='100px'))
close_ts_butt       = wg.Button(description="Close Graph", icon = "times")
clear_list_butt     = wg.Button(description="Clear List", icon = "times")
ts_sites            = wg.SelectMultiple(options=[], value=[], description='Site List:')
error_bar_check     = wg.Checkbox(description="Error Bars", value=False)
detrend_check       = wg.Checkbox(description="Detrend", value=False)
live_update_check   = wg.Checkbox(description="Live Update", value=False)
start_year_form     = wg.Text(value=None,placeholder='YYYY-MM-DD',description='Start:',disabled=False, style= {'description_width': 'initial'}, layout=layout("150px"))
end_year_form       = wg.Text(value=None,placeholder='YYYY-MM-DD',description='End:',disabled=False, style= {'description_width': 'initial'}, layout=layout("150px"))
siglim_form         = wg.Text(value=None,placeholder='(10,10,30)',description='SigLim:',disabled=False, style= {'description_width': 'initial'}, layout=layout("150px"))
# Style
append_butt.style.button_color = 'rgb(196,253,196)'
plot_ts_butt.style.button_color = 'paleturquoise'
close_ts_butt.style.button_color = 'mistyrose'
clear_list_butt.style.button_color = 'mistyrose'
# Outputs
timeseries_output   = wg.Output()

### Widget Functions

#### Fetcher

In [ ]:
def UpdateUNR(_):
    with fetcher_output:
        print("Downloading Data from UNR...")

    plates_site = "http://geodesy.unr.edu/gps_timeseries/sta_frames.txt"
    coords_site = "http://geodesy.unr.edu/NGLStationPages/llh.out"
    
    plates_list = str(urllib.request.urlopen(plates_site).read())[2:].split(" \\n")[:-1]
    coords_list = str(urllib.request.urlopen(coords_site).read())[2:].split(" \\n")[:-1]
    
    for_json = {}

    for siteplate in plates_list:
        siteplates = [el for el in siteplate.split(" ") if el]
        for_json.setdefault(siteplates[0], {'location': None, 'networks': []})
        for_json[siteplates[0]]['networks'] = siteplates[1:]
    
    for sitecoord in coords_list:
        siteid, lon, lat, height = (el for el in sitecoord.split(" ") if el)
        for_json.setdefault(siteid, {'location': None, 'networks': []})
        for_json[siteid]['location'] = [float(lon), float(lat)]
    
    global data_of
    data_of["UNR"] = for_json

    with open('./data/UNR_data.json', 'w') as f:
        json.dump(for_json, f)
        
    with fetcher_output:
        print("Latest UNR Data Downloaded")

update_UNR_butt.on_click(UpdateUNR)


def UpdateJPL(_):
    with fetcher_output:
        print("Downloading Data from JPL...")

    coords_site = "https://sideshow.jpl.nasa.gov/post/tables/table2.html"
    x = str(urllib.request.urlopen(coords_site).read()).split("\\n")
    sitecoordlist = [[xxx for xxx in xx.split(" ") if xxx]
                     for xx in x if "POS" in xx]
    sitevellist = [[xxx for xxx in xx.split(" ") if xxx]
                     for xx in x if "VEL" in xx]
    # [['AB01', 'POS', '52.2095', '-174.2048', '25492.217', '0.034', '0.024', '0.098'], ...]
    for_json = {}
    for element in sitecoordlist:
        for_json[element[0]] = {'location': [float(element[2]),float(element[3])]}
    for element in sitevellist:
        for_json[element[0]]['velocity'] = [float(element[2]),float(element[3])]
    
    global data_of
    data_of["JPL"] = for_json

    with open('./data/JPL_data.json', 'w') as f:
        json.dump(for_json, f)
    
    with fetcher_output:
        print("Latest JPL Data Downloaded")
update_JPL_butt.on_click(UpdateJPL)


def UpdateUNAV(_):
    with fetcher_output:
        print("Downloading Data from UNAV...")

    coords_site = "https://www.unavco.org/instrumentation/networks/status/data/geoJSON/network-monitoring"
    
    x = json.loads(urllib.request.urlopen(coords_site).read())

    # [['AB01', 'POS', '52.2095', '-174.2048', '25492.217', '0.034', '0.024', '0.098'], ...]
    for_json = {}
    for element in x['features']:
        for_json[element["id"]] = {
            'location': [element['geometry']['coordinates'][1],element['geometry']['coordinates'][0]], 
            'region': element['properties']['region']}
    
    global data_of
    data_of["UNAV"] = for_json

    with open('./data/UNAV_data.json', 'w') as f:
        json.dump(for_json, f)
    
    with fetcher_output:
        print("Latest UNAV Data Downloaded")
update_UNAV_butt.on_click(UpdateUNAV)

#### Availability

In [ ]:
def site_search(b):
    searched = site_searchbar.value.upper() 
    with availability_output:
        try:
            print(searched + "(" + org_avail_select.value + ")", ": ", data_of[org_avail_select.value][searched])
        except:
            display(wg.HTML('''<em style="color:red">Not Found!</em>'''))
site_search_submit.on_click(site_search)


def clear_avail_log(b):
    availability_output.clear_output()
clear_log_butt.on_click(clear_avail_log)

#### Map

In [ ]:
def new_map(b):
    map_output.clear_output()
    map[0] = fol.Map(control_scale=True,
                   location=(40, -100),
                   zoom_start=4,
                   width = 1100,
                   height = 600)
    with map_output:
        display(map[0])
new_map_butt.on_click(new_map)

def reload_map(b):
    map_output.clear_output()
    with map_output:
        display(map[0])
reload_map_butt.on_click(reload_map)

def close_map(b):
    map_output.clear_output()
    nearest_site_output.clear_output()
    graph_output.clear_output()
    map[0] = "Not initialized yet!"
close_map_butt.on_click(close_map)

################
# SITE BUTTONS #
################

def site_circle(b):
    siteid, rad = site_center.value, float(site_radius.value) if site_radius.value else None
    plotvec = plot_vec_check.value
    
    if rad:
        site_list, center = nearby_sites(data_of[org_map_select.value], siteid, rad)
        basic_circle(map[0], center, rad)
        plot_locations(map[0], [el[1:3] for el in site_list], [el[0] for el in site_list])
        
        if plotvec:
            for el in site_list:
                plot_velocity(el[0], org_map_select.value)
    
    if siteid[0] in "[(":
        siteid = list(eval(siteid))
    
    plot_locations(map[0], 
                   [data_of[org_map_select.value][siteid]["location"] if isinstance(siteid, str) else siteid], 
                   [siteid], color="black", dotrad = 4)
    
    if plotvec:
        plot_velocity(siteid, org_map_select.value)

    reload_map(b)
site_circle_submit.on_click(site_circle)


def nn_graph(b):
    siteid, rad = site_center.value, float(site_radius.value) if site_radius.value else None

    if not rad:
        nearest_site_output.clear_output()
        graph_output.clear_output()
        return

    site_list, _ = nearby_sites(data_of[org_map_select.value], siteid, rad)
    neighbours = sorted(site_list, key = lambda x: x[3])[:10]
    
    nearest_site_output.clear_output()
    with nearest_site_output:
        display(wg.HTML("""<h2>10 Nearest Neighbouring Sites:</h2>"""))
        tableHTML = f"<table><tr><th><b>Site</b></th><th><b>Distance from {siteid}</b></th></tr>"
        for s,_,_,d in neighbours: tableHTML += f"<tr><td>{s}</td><td>{d:.3f} km</td></tr>"
        display(wg.HTML(tableHTML + "</table>"))
        
    graph_output.clear_output()
    with graph_output:
        display(wg.HTML("""<h2 style="text-align:center;">Distances to Nearby Sites (km)</h2>"""))
        X = range(len(neighbours))
        plt.bar([x[0] for x in neighbours], [x[3] for x in neighbours])
        plt.show()
plot_graph_butt.on_click(nn_graph)


def get_vel(siteid, org):
    try:
        return data_of[org][siteid]["velocity"]
    except:
        times, tseries = Read[org](siteid, 0)
        td, nd, ns, ed, es, ud, us = tseries
        return [FitTS(np.array(td), np.array(nd), np.array(ns))[1][0], 
                FitTS(np.array(td), np.array(ed), np.array(es))[1][0]]


def plot_velocity(siteid, org):
    xdot,ydot = get_vel(siteid, org)
    data_of[org][siteid]["velocity"] = [xdot, ydot]
    draw_vector(map[0], data_of[org][siteid]["location"], [xdot, ydot], 0.2, color="black")
# enter_code_butt.on_click(plot_velocity)


#### Timeseries

In [ ]:
def update_ts_list(x):
    siteid = ts_site_form.value
    
    # check validity
    if siteid not in data_of[org_ts_select.value]:
        timeseries_output.clear_output()
        with timeseries_output:
            display(wg.HTML('''<em style="color:red">Invalid site! Try again.</em>'''))
    else:
        extendedsiteid = siteid + " (" + org_ts_select.value + ")"
        if extendedsiteid not in ts_sites.options:
            ts_sites.options = list(ts_sites.options) + [extendedsiteid]
        
    ts_site_form.value = ""
    
append_butt.on_click(update_ts_list)

def list_to_graph(x):
    selected = list(ts_sites.value)
    timeseries_output.clear_output()

    if siglim_form.value:
        global SigLim
        try: 
            nlim, elim, ulim = eval(siglim_form.value)
            assert all(isinstance(lim,int) for lim in [nlim, elim, ulim])
        except:
            nlim, elim, ulim = 10, 10, 30
        SigLim = (nlim, elim, ulim)
    
    # check validity
    
    yearrange = [datetime(1,1,1), datetime(3000,1,1)] # [far past, far future]

    if selected:
        if start_year_form.value: yearrange[0] = datetime.strptime(start_year_form.value, "%Y-%m-%d")
        if end_year_form.value: yearrange[1] = datetime.strptime(end_year_form.value, "%Y-%m-%d")

        with timeseries_output:
            plot_ts_graph_list(
                selected, 
                int(live_update_check.value), 
                yearrange, 
                detrend = detrend_check.value, 
                errorbars = error_bar_check.value,
                resolution = plot_ts_res.value
                )
    else:
        with timeseries_output:
            display(wg.HTML('''<em style="color:red">Select a set of sites!</em>'''))

    ts_site_form.value = ""

plot_ts_butt.on_click(list_to_graph)

def close_ts(b):
    timeseries_output.clear_output()
close_ts_butt.on_click(close_ts)

def clear_list(b):
    ts_sites.options = []
clear_list_butt.on_click(clear_list)

### Windows

In [ ]:
fetch_window = VBox([
    wg.HTML(
            '<em style="color:blue; line-height: 1.5">Use the buttons below to update the local files with the latest data. This may take a few seconds.</em>'
            ),
    HBox([update_UNR_butt, update_JPL_butt, update_UNAV_butt]),
    fetcher_output
])

availability_window = VBox([
    wg.HTML('<h1>Availability</h1>'),
    wg.HTML(
            '<em style="color:blue; line-height: 1.5"><ul>' + 
            '<li>Type the Site ID into the box below to check its existence and status.</li>' +
            '</ul></em>'
            ),
    HBox([site_searchbar, org_avail_select, site_search_submit, clear_log_butt]),
    availability_output
    ])

map_window = VBox([
    wg.HTML('<h1>Map</h1>'),
    wg.HTML(
        '<em style="color:blue; line-height: 1.5"><ul>' + 
        '<li>Click New Map to initialize a new map that you can plot on. You can click on this button any time to reset a new map.</li>' +
        '<li>Type the Site ID and the radius within which you want to detect other sites. Click Add Plot to overlay the circle.</li>' +
        '<li>Click Plot Distances to show the top 10 nearest sites.</li>' +
        '</ul></em>'
    ),
    HBox([new_map_butt, reload_map_butt, close_map_butt, enter_code_form, plot_vec_check]),
    HBox([site_center, org_map_select, site_radius, site_circle_submit, plot_graph_butt]),
    HBox([nearest_site_output, graph_output]),
    map_output
    ])

timeseries_window = VBox([
    wg.HTML('<h1>Timeseries</h1>'),
    wg.HTML(
        '<em style="color:blue; line-height: 1.5"><ul>' + 
        '<li>Input a Site ID to add it to the list.</li>' +
        '<li>Select multiple sites from the list by holding cmd/ctrl to plot multiple sites. Click Plot to plot the timeseries.</li>' +
        '<li>Plotting parameters:<ul>' + 
        '<li>Check Error Bars to show error bars. </li>' +
        '<li>Check Detrend to plot a linearly detrended timeseries. </li>' + 
        '<li>Check Live Update to download the latest timeseries of all the sites chosen before plotting.<br>(Without Live Update, the latest timeseries is downloaded only if it is not available locally.) </li>' + 
        '<li>Adjust SigLim = (nlim, elim, ulim) to remove NEU with sigmas > nlim, elim, ulim. By default, SigLim = (10,10,30).</li>' + 
        '<li>Use Start/End Year to restrict the time series to that year range.</li>'
        '</ul></li>' +
        '</ul></em>'
    ),
    HBox([ts_site_form, org_ts_select, append_butt]),
    HBox([start_year_form, end_year_form, siglim_form, error_bar_check, detrend_check, live_update_check]),
    HBox([ts_sites,VBox
          ([HBox([clear_list_butt, close_ts_butt]),
            HBox([plot_ts_butt, plot_ts_res])])
        ]),
    timeseries_output
    ])

## Interface

To open the interface in the browser using the Voila package, comment the first line and then uncomment the rest.

In [ ]:
# display(fetch_window, availability_window, map_window, timeseries_window)

import nbformat as nbf

newcell = nbf.notebooknode.NotebookNode(
    cell_type='code',
    execution_count = 0,
    metadata = {},
    outputs= [],
    source='display(fetch_window, availability_window, map_window, timeseries_window)')

ntbk = nbf.read("main_project.ipynb", nbf.NO_CONVERT)
ntbk.cells = [cell for cell in ntbk.cells if cell.cell_type != "markdown"][:-1] + [newcell]

nbf.write(ntbk, "temp.ipynb", version=nbf.NO_CONVERT)

! voila temp.ipynb